<a href="https://colab.research.google.com/github/rmngllnn/Stage2021/blob/main/POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# POS-Tagging  

Ce notebook permet :  
- d'extraire les données d'un fichier CSV  
- Nettoyer les données. 
- POS-Tagger les phrases extraites  
- Obtenir la liste des noms, adjectifs et pronoms

In [ ]:
google_colab = False

In [ ]:
import pandas as pd
import spacy
import csv
import json

from ipynb.fs.defs.fonctions_preprocess import *

## Aperçu des données

In [ ]:
df = pd.read_csv("../DEMOCRATIE_ET_CITOYENNETE.csv", low_memory = False)
df.shape

#df.head()

## Extraction et Nettoyage des données

https://docs.python.org/fr/3/library/csv.htmlhttps://docs.python.org/fr/3/library/csv.html

### Extraction des données

In [ ]:
#Extraction des données dans une liste de listes
#len(ex) = 28385 soit le nombre d’observation

csv.field_size_limit(100000000) # Adapte la taille limite autorisée

data = open_doc_csv ("../DEMOCRATIE_ET_CITOYENNETE.csv")

In [ ]:
print(len(data))

In [ ]:
for obs in data[1500:1700]:
    print(obs)

### Nettoyage des données

In [ ]:
cleaned_data = clean_data(data)

In [ ]:
for obs in cleaned_data[1500:1800]:
    print(obs)

In [ ]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

In [ ]:
serialisation_data(cleaned_data, "all_data_cleaned.json")

In [ ]:
#Runner que cette cellule si les données viennent d'un fichier json

data = open_file("all_data_cleaned.json")

In [ ]:
print(len(data))
print(type(data))

## POS-Tagging using SpaCy

In [ ]:
#NE PAS RUN cette cellule sauf sur Google Colab
#Permet d'installer le module sur le serveur distant utilisé.

if google_colab:
  try:
    nlp = spacy.load("fr_core_news_lg")
  except:
    from spacy.cli import download
    download('fr_core_news_sm')
    nlp = spacy.load("fr_core_news_lg")

In [ ]:
nlp = spacy.load("fr_core_news_lg")

In [ ]:
#doc = nlp(d) # /!\ Doit être de type String

def pos_tagger(data_list):
    
    """ Tag each token of data in input
    -> list of sentences
    <- lists of nouns, adjectives and pronouns with lemma of each token
    """
  
    nouns = []
    adj = []
    pronouns = []
    
    loop = 100000
    
    def tag(data_item):
        doc = nlp(data_item)
        for token in doc:
            if token.pos_ == "NOUN" : nouns.append((token.text, token.lemma_))
            elif token.pos_ == "ADJ" : adj.append((token.text, token.lemma_))
            elif token.pos_ == "PRON" : pronouns.append((token.text, token.lemma_))
                
    for i in range (0, len(data_list)-loop, loop):
        print("Nous en sommes à" , i, "/", len(data_list))
        for j in range (i, i+loop):
            tag(data_list[j])
    print("Nous en sommes à" , loop*(len(data_list)//loop), "/", len(data_list))
    for j in range(loop*(len(data_list)//loop), len(data_list)):
        tag(data_list[j])
    
    return nouns, adj, pronouns

#### Séparation en trois groupes pour faciliter les calculs

In [ ]:
print("tag slice 1")
tags_1 = pos_tagger(cleaned_data[0:1000000])
serialisation_data(tags_1, "pos1.json")

In [ ]:
print("slice tag 2")
tags_2 = pos_tagger(data[1000000:2000000])
serialisation_data(tags_2, "pos2.json")
print("DONE !")

In [ ]:
print(tags_2[0][0:10])

In [ ]:
print("tag slice 3")
tags_3 = pos_tagger(data[2000000:len(data)])
serialisation_data(tags_3, "pos3.json")
print("DONE !")

#### Récupération des trois fichiers et concaténation des listes pour avoir trois listes : nouns, pronouns, adjectives

In [ ]:
pos1 = open_file("pos1.json")
print(len(pos1))
print(len(pos1[0]))
print(len(pos1[1]))
print(len(pos1[2]))

In [ ]:
pos2 = open_file("pos2.json")
print(len(pos2))

In [ ]:
pos3 = open_file("pos3.json")
print(len(pos3))

In [ ]:
print(len(pos1[0]+pos2[0]+pos3[0]))
print(len(pos1[1]+pos2[1]+pos3[1]))
print(len(pos1[2]+pos2[2]+pos3[2]))

In [ ]:
nouns = pos1[0] + pos2[0] + pos3[0]

#### Sérialisation des trois listes

In [ ]:
serialisation_data(nouns, "nouns.json")

## Sérialisation des données

Données sérialisées au format json car facilement lisible hors programmation  

https://www.codeflow.site/fr/article/python-json  

La cellule ci-dssous ne doit être runé qu'une seule fois

In [ ]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

In [ ]:
serialisation_data(a, "test.json")